In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import pdal
from dask.distributed import Client

np.set_printoptions(precision=4, suppress=True)

from forest_structure_tools.metrics import forest_structure_metrics

In [2]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
plots_lidar_dir = plots_dir / "lidar"

plots_gdf = gpd.read_file(plots_dir / "plots.geojson")
plot_ids = plots_gdf['id'].to_list()

def read_plot_lidar(plot_id: str):
    pl = pdal.Reader(str(plots_lidar_dir / f"{plot_id}.copc.laz")).pipeline()
    pl.execute()

    return pl.arrays[0]

In [3]:
metrics_dir = plots_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

In [20]:
def add_plot_dimension_to_ds(ds: xr.Dataset, plot_id: str) -> xr.Dataset:
    site = plot_id[:-3]
    site_type = site[:3]

    ds = ds.expand_dims(plot=[plot_id]).assign_coords(
        plot=("plot", [plot_id]), site=("plot", [site]), site_type=("plot", [site_type])
    )

    return ds


def calculate_metrics(plot_id: id, xy_bin_size: float | None = None, z_bin_size: float | None = 1, z_bin_count: float | None = 10, weighted = False):
    points = read_plot_lidar(plot_id)
    x = points["X"]
    y = points["Y"]
    z = points["Z"]

    if weighted:
        weights = 1 / points["NumberOfReturns"]
    else:
        weights = None

    metrics = forest_structure_metrics(x=x, y=y, z=z, xy_bin_size=xy_bin_size, z_bin_size=z_bin_size, z_bin_count=z_bin_count, weights=weights)


    if xy_bin_size is not None:
        x_offset = metrics.x.min().item()
        y_offset = metrics.y.min().item()

        metrics['x_offset'] = x_offset
        metrics['y_offset'] = y_offset

        metrics['x'] = metrics['x'] - x_offset
        metrics['y'] = metrics['y'] - y_offset


    metrics = add_plot_dimension_to_ds(metrics, plot_id)

    metrics.attrs["xy_bin_size"] = str(xy_bin_size)
    metrics.attrs["z_bin_size"] = str(z_bin_size)
    metrics.attrs["weighted"] = str(weighted)
    
    return metrics


In [22]:
calculate_metrics("AGG_O_01_P1", xy_bin_size=None, z_bin_size=1, z_bin_count=50, weighted=False)

<xarray.Dataset> Size: 3kB
Dimensions:              (plot: 1, z: 18, qz: 51)
Coordinates:
  * z                    (z) float64 144B 0.0 1.0 2.0 3.0 ... 15.0 16.0 17.0
  * qz                   (qz) float64 408B 0.0 2.0 4.0 6.0 ... 96.0 98.0 100.0
  * plot                 (plot) <U11 44B 'AGG_O_01_P1'
    site                 (plot) <U8 32B 'AGG_O_01'
    site_type            (plot) <U3 12B 'AGG'
Data variables: (12/46)
    max                  (plot) float64 8B 16.28
    min                  (plot) float64 8B 0.0
    range                (plot) float64 8B 16.28
    mean                 (plot) float64 8B 4.428
    median               (plot) float64 8B 4.555
    sd                   (plot) float64 8B 4.093
    ...                   ...
    qz_pai               (plot) float64 8B 1.533
    qz_fhd               (plot) float64 8B 3.175
    qz_norm_fhd          (plot) float64 8B 0.8076
    qz_cv_inside_p       (plot) float64 8B 1.757
    qz_cv_ppi            (plot) float64 8B 0.1525
    qz_cv_pad            (plot) float64 8B 2.304
Attributes:
    xy_bin_size:  None
    z_bin_size:   1
    weighted:     False

In [23]:
def calculate_and_save_metrics(plot_ids : list[str], client : Client, xy_bin_size: float | None = None, z_bin_size: float | None = 1, z_bin_count: float | None = 10, weighted = False):
    futures = client.map(calculate_metrics, plot_ids, xy_bin_size=xy_bin_size, z_bin_size=z_bin_size, z_bin_count=z_bin_count, weighted=weighted, retries=5)
    results = client.gather(futures)
    ds = xr.concat(results, dim='plot')

    xy = f'{xy_bin_size}m' if xy_bin_size is not None else 'none'
    z = f'{z_bin_size}m' if z_bin_size is not None else 'none'
    w = '_w' if weighted else ''

    ds.to_netcdf(metrics_dir / f"metrics_grid_{xy}_z_{z}{w}.nc")

In [18]:
client = Client()  # Start a Dask client
client

/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60036 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60036/status,
Dashboard: http://127.0.0.1:60036/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60037,Workers: 0
Dashboard: http://127.0.0.1:60036/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:60051,Total threads: 2
Dashboard: http://127.0.0.1:60053/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:60040,


In [24]:
# XY None
calculate_and_save_metrics(plot_ids, client, xy_bin_size=None, z_bin_size=1, z_bin_count=50, weighted=False)

In [25]:
# XY None Weighted
calculate_and_save_metrics(plot_ids, client, xy_bin_size=None, z_bin_size=1, weighted=True)

In [10]:
# XY 5m
calculate_and_save_metrics(plot_ids, client, xy_bin_size=5, z_bin_size=1, weighted=False)

In [13]:
# XY 5m Weighted
calculate_and_save_metrics(plot_ids, client, xy_bin_size=5, z_bin_size=1, weighted=True)

In [8]:
# XY 10m
calculate_and_save_metrics(plot_ids, client, xy_bin_size=10, z_bin_size=1, weighted=False)
calculate_and_save_metrics(plot_ids, client, xy_bin_size=10, z_bin_size=1, weighted=True)

In [17]:
# XY 20m
calculate_and_save_metrics(plot_ids, client, xy_bin_size=20, z_bin_size=1, weighted=False)
calculate_and_save_metrics(plot_ids, client, xy_bin_size=20, z_bin_size=1, weighted=True)

In [ ]:
# client.close()